In [1]:
import pandas as pd
import os
from rapidfuzz import fuzz
import re
from unidecode import unidecode


In [2]:
# Loading csv files into dataframes
player_stats_df = pd.read_csv('./data/Player_stats/all_players.csv')
keeper_stats_df = pd.read_csv('./data/Player_stats/all_goalkeepers.csv')

In [3]:
# PKG/A needs to be changed. 
keeper_stats_df.head()

,Player,Club,POS,GP,GS,MINS,SHTS,SV,GA,GAA,PKG/A,W,L,T,ShO,W%,Sv%,Year,Season
0,Mark Dodd,DAL,GK,31,31,2776,235,160,45,1.45,5/5,17,14,0,6,54.8,68.1,1996,reg
1,Tony Meola,MET,GK,29,29,2610,188,142,38,1.31,3/4,14,15,0,9,48.3,75.5,1996,reg
2,Mark Dougherty,TB,GK,28,28,2520,186,123,47,1.68,9/10,17,11,0,3,60.7,66.1,1996,reg
3,Jorge Campos,LA,GK,24,24,2160,133,100,27,1.13,0/1,13,8,0,4,54.2,75.2,1996,reg
4,Garth Lagerwey,KC,GK,23,22,1959,133,83,38,1.73,2/2,12,10,0,3,52.2,62.4,1996,reg


In [4]:
# Will be dropping Club, so no nulls to deal with in this dataframe
keeper_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2006 entries, 0 to 2005
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  2006 non-null   object 
 1   Club    1324 non-null   object 
 2   POS     2006 non-null   object 
 3   GP      2006 non-null   int64  
 4   GS      2006 non-null   int64  
 5   MINS    2006 non-null   int64  
 6   SHTS    2006 non-null   int64  
 7   SV      2006 non-null   int64  
 8   GA      2006 non-null   int64  
 9   GAA     2006 non-null   float64
 10  PKG/A   2006 non-null   object 
 11  W       2006 non-null   int64  
 12  L       2006 non-null   int64  
 13  T       2006 non-null   int64  
 14  ShO     2006 non-null   int64  
 15  W%      2006 non-null   float64
 16  Sv%     2006 non-null   float64
 17  Year    2006 non-null   int64  
 18  Season  2006 non-null   object 
dtypes: float64(3), int64(11), object(5)
memory usage: 297.9+ KB


In [5]:
keeper_stats_df.describe()

,GP,GS,MINS,SHTS,SV,GA,GAA,W,L,T,ShO,W%,Sv%,Year
count,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000
mean,6.804586,6.677966,603.997009,33.926720,23.310568,9.608175,0.888500,2.629611,2.626620,1.420738,1.606680,18.861316,37.418096,2010.110668
std,10.316232,10.292286,924.747438,52.330135,36.569597,14.673335,0.992154,4.433369,4.090945,2.753633,2.855163,24.494023,34.628706,7.284097
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1996.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2004.000000
50%,1.000000,1.000000,90.000000,7.000000,4.000000,2.000000,0.940000,0.000000,1.000000,0.000000,0.000000,0.000000,52.700000,2011.000000
75%,9.000000,8.000000,770.250000,47.000000,31.000000,13.750000,1.500000,3.000000,3.000000,1.000000,2.000000,37.500000,68.900000,2016.750000
max,34.000000,34.000000,3060.000000,249.000000,184.000000,64.000000,6.000000,22.000000,19.000000,18.000000,16.000000,100.000000,100.000000,2020.000000


In [6]:
player_stats_df.head()

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,...,RdA,A/90min,FC,FS,OFF,YC,RC,SOG%,Year,Season
0,Roy Lassiter,TB,F,30,30,2580,27,4,76,49,...,2,0.14,20,39,70,2,0,64.47,1996,reg
1,Raul Diaz Arce,DC,F,28,28,2351,23,2,100,49,...,2,0.08,32,26,35,6,1,49.00,1996,reg
2,Eduardo Hurtado,LA,F,26,26,2323,21,7,87,56,...,3,0.27,48,26,25,5,0,64.37,1996,reg
3,Preki,KC,M,32,32,2880,18,13,140,61,...,4,0.41,26,44,7,3,0,43.57,1996,reg
4,Brian McBride,CLB,F,28,28,2307,17,3,79,44,...,1,0.12,21,46,10,0,0,55.70,1996,reg


In [7]:
#A lot of columns to sift through, but no nulls to deal with since we will be dropping club. 
player_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15767 entries, 0 to 15766
Data columns (total 28 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Player   15767 non-null  object 
 1   Club     10274 non-null  object 
 2   POS      15767 non-null  object 
 3   GP       15767 non-null  int64  
 4   GS       15767 non-null  int64  
 5   MINS     15767 non-null  int64  
 6   G        15767 non-null  int64  
 7   A        15767 non-null  int64  
 8   SHTS     15767 non-null  int64  
 9   SOG      15767 non-null  int64  
 10  GWG      15767 non-null  int64  
 11  PKG/A    15767 non-null  object 
 12  HmG      15767 non-null  int64  
 13  RdG      15767 non-null  int64  
 14  G/90min  15767 non-null  float64
 15  SC%      15767 non-null  float64
 16  GWA      15767 non-null  int64  
 17  HmA      15767 non-null  int64  
 18  RdA      15767 non-null  int64  
 19  A/90min  15767 non-null  float64
 20  FC       15767 non-null  int64  
 21  FS       157

In [8]:
#Creating a list of the years we want to use from the dataframes
years = [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

In [9]:
#Filtering dataframes to only show years in the years list
player_stats_df = player_stats_df[player_stats_df['Year'].isin(years)]
keeper_stats_df = keeper_stats_df[keeper_stats_df['Year'].isin(years)]

In [10]:
player_stats_df

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,...,RdA,A/90min,FC,FS,OFF,YC,RC,SOG%,Year,Season
3029,Luciano Emilio,DC,F,29,28,2410,20,1,79,47,...,0,0.04,35,39,21,2,0,59.49,2007,reg
3030,Juan Pablo Angel,NY,F,24,24,2125,19,5,97,53,...,2,0.21,31,20,42,2,1,54.64,2007,reg
3031,Taylor Twellman,NE,F,26,25,2283,16,3,90,55,...,1,0.12,13,41,21,2,0,61.11,2007,reg
3032,Eddie Johnson,KC,F,24,24,2149,15,6,75,43,...,3,0.25,21,57,50,6,0,57.33,2007,reg
3033,Maykel Galindo,CHV,F,28,24,2021,12,5,55,28,...,2,0.22,44,53,30,6,0,50.91,2007,reg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15762,Alejandro Pozuelo,NaN,M,1,1,120,0,0,5,0,...,0,0.00,2,0,0,0,0,0.00,2020,post
15763,Franco Jara,NaN,F,2,2,210,0,0,5,1,...,0,0.00,6,2,3,0,0,20.00,2020,post
15764,Andy Polo,NaN,M,1,1,105,0,0,6,3,...,0,0.00,0,3,0,0,0,50.00,2020,post
15765,Cristian Roldan,NaN,M,4,4,360,0,0,6,1,...,0,0.00,5,11,0,0,0,16.67,2020,post


In [11]:
player_stats_df['Season'].value_counts()

Season
reg     7245
post    3727
Name: count, dtype: int64

In [12]:
# Filtering dataframes to only have data from the regular season
player_stats_df = player_stats_df[player_stats_df['Season'] == 'reg']
keeper_stats_df = keeper_stats_df[keeper_stats_df['Season'] == 'reg']

In [13]:
player_stats_df

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,...,RdA,A/90min,FC,FS,OFF,YC,RC,SOG%,Year,Season
3029,Luciano Emilio,DC,F,29,28,2410,20,1,79,47,...,0,0.04,35,39,21,2,0,59.49,2007,reg
3030,Juan Pablo Angel,NY,F,24,24,2125,19,5,97,53,...,2,0.21,31,20,42,2,1,54.64,2007,reg
3031,Taylor Twellman,NE,F,26,25,2283,16,3,90,55,...,1,0.12,13,41,21,2,0,61.11,2007,reg
3032,Eddie Johnson,KC,F,24,24,2149,15,6,75,43,...,3,0.25,21,57,50,6,0,57.33,2007,reg
3033,Maykel Galindo,CHV,F,28,24,2021,12,5,55,28,...,2,0.22,44,53,30,6,0,50.91,2007,reg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15296,Sebastian Mendez,ORL,M,19,11,1087,0,0,17,3,...,0,0.00,32,9,1,4,1,17.65,2020,reg
15297,Kelyn Rowe,NE,M,16,10,896,0,0,17,4,...,0,0.00,13,4,0,3,0,23.53,2020,reg
15298,Siem de Jong,CIN,M,15,8,793,0,0,18,5,...,0,0.00,7,8,1,0,0,27.78,2020,reg
15299,Maxime Chanot,NYC,D,20,20,1712,0,0,19,4,...,0,0.00,18,6,1,5,1,21.05,2020,reg


In [14]:
keeper_stats_df

,Player,Club,POS,GP,GS,MINS,SHTS,SV,GA,GAA,PKG/A,W,L,T,ShO,W%,Sv%,Year,Season
427,Bouna Coundoul,COL,GK,30,30,2668,158,120,32,1.07,2/2,9,12,8,9,30.0,75.9,2007,reg
428,Kevin Hartman,KC,GK,30,30,2700,159,110,45,1.50,6/7,11,12,7,5,36.7,69.2,2007,reg
429,Matt Reis,NE,GK,30,30,2700,169,120,43,1.43,3/3,14,8,8,10,46.7,71.0,2007,reg
430,Joe Cannon,LA,GK,29,29,2610,171,119,46,1.59,4/5,9,13,7,5,31.0,69.6,2007,reg
431,Troy Perkins,DC,GK,29,29,2610,155,117,32,1.10,1/3,16,6,7,8,55.2,75.5,2007,reg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1940,Emmanuel Ochoa,SJ,GK,0,0,0,0,0,0,0.00,0/0,0,0,0,0,0.0,0.0,2020,reg
1941,Drake Callender,MIA,GK,0,0,0,0,0,0,0.00,0/0,0,0,0,0,0.0,0.0,2020,reg
1942,Elliot Panicco,NSH,GK,0,0,0,0,0,0,0.00,0/0,0,0,0,0,0.0,0.0,2020,reg
1943,Eric Dick,SKC,GK,0,0,0,0,0,0,0.00,0/0,0,0,0,0,0.0,0.0,2020,reg


In [15]:
# Loading in all the salary datasets into respective dataframes
salary2007_df = pd.read_excel('./data/Salary_data/2007-Salaries.xlsx', header=0)
salary2008_df = pd.read_excel('./data/Salary_data/2008-Salaries.xlsx', header=0)
salary2009_df = pd.read_excel('./data/Salary_data/2009-Salaries.xlsx', header=0)
salary2010_df = pd.read_excel('./data/Salary_data/2010-Salaries.xlsx', header=0)
salary2011_df = pd.read_excel('./data/Salary_data/2011-Salaries.xlsx', header=1)
salary2012_df = pd.read_excel('./data/Salary_data/2012-Salaries.xlsx', header=1)
salary2013_df = pd.read_excel('./data/Salary_data/2013-Salaries.xlsx', header=1)
salary2014_df = pd.read_excel('./data/Salary_data/2014-Salaries.xlsx', header=1)
salary2015_df = pd.read_excel('./data/Salary_data/2015-Salaries.xlsx', header=1)
salary2016_df = pd.read_excel('./data/Salary_data/2016-Salaries.xlsx', header=1)
salary2017_df = pd.read_excel('./data/Salary_data/2017-Salaries.xlsx', header=1)
salary2018_df = pd.read_excel('./data/Salary_data/2018-Salaries.xlsx', header=0)
salary2019_df = pd.read_excel('./data/Salary_data/2019-Salaries.xlsx', header=1)
salary2020_df = pd.read_excel('./data/Salary_data/2020-Salaries-.xlsx', header=0)


In [16]:
salary2007_df.head()

,Club,Last Name,First Name,Pos,Base Salary,Compensation
0,DC,Addlery,Nicholas,F,36000,36000
1,LA,Albright,Chris,M-D,142500,142500
2,NY,Altidore,Josmer,F,85000,108333
3,DAL,Alvarez,Arturo,M,32819,46019
4,NY,Angel,Juan Pablo,F,1500000,1593750


In [17]:
salary2008_df.head()

,Club,Last Name,First Name,Pos,Base Salary,Compensation
0,LA,Adzemian,Vardan,D,12900,12900
1,LA,Alamo,Charles,GK,12900,12900
2,NE,Albright,Chris,D,160000,160000
3,HOU,Alcaraz,Johnny,M,12900,12900
4,LA,Allen,Ely,F,12900,12900


In [18]:
salary2009_df.head()

,Club,Last Name,First Name,Pos,Base Salary,Compensation
0,NE,Albright,Chris,D,176000,176000
1,RSL,Alexandre,Jean,M,34000,34000
2,DC,Allen,Ely,F-M,15300,15300
3,SEA,Alonso,Osvaldo,M,65000,65000
4,NE,Alston,Kevin,D,55000,104000


In [19]:
salary2010_df.head()

,Club,Last Name,First Name,Pos,Base Salary,Compensation
0,NY,Agudelo,Juan,F,40000,70000
1,KC,Aiyegbusi,Olukorede,D,40000,40000
2,COL,Akpan,Andre,F,40000,65000
3,NY,Albright,Chris,D,84000,84000
4,DAL,Alexander,Eric,M,40000,40000


In [20]:
salary2011_df.head()

,Club,Last Name,First Name,Pos,Base Salary,Compensation
0,COL,Ababio,Eddie,D,32604,32604
1,PHI,Adu,Freddy,M,475884,594884
2,RSL,Agorsor,Christopher,F,32604,32604
3,NY,Agudelo,Juan,F,55000,85000
4,KC,Aiyegbusi,Olukorede,D,42000,42000


In [21]:
# What is Unnamed:5 ?
salary2020_df.head()

,Last Name,First Name,2020 Fall Salary List Club,Playing Position,Base Salary,Unnamed: 5,Base Guaranteed Comp.
0,Aaronson,Brenden,Philadelphia Union,M,85000,NaN,103309
1,Abdul-Salaam,Saad,FC Cincinnati,D,81375,NaN,90042
2,Abu,Mohammed,DC United,M,81375,NaN,81375
3,Abubakar,Lalas,Colorado Rapids,D,145000,NaN,154937
4,Accam,David,Nashville SC,F,875000,NaN,1002916


In [22]:
#These are salaries. Went back into the original excel spreadsheet that the dataset is on and found that at one point
#the information for Base Salary gets split into two columns.
salary2020_df['Unnamed: 5'].value_counts()

Unnamed: 5
81375.0      8
63547.0      8
100000.0     2
155004.0     1
800000.0     1
192500.0     1
315000.0     1
525000.0     1
1457148.0    1
275000.0     1
6000000.0    1
750000.0     1
185000.0     1
500000.0     1
85000.0      1
200004.0     1
151255.0     1
166000.0     1
730008.0     1
210000.0     1
Name: count, dtype: int64

In [23]:
#filling rows with  'Base Salary' nulls with the filled in 'Unnamed: 5' salary values
salary2020_df['Base Salary'].fillna(salary2020_df['Unnamed: 5'], inplace=True)
salary2020_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Last Name                   795 non-null    object 
 1   First Name                  800 non-null    object 
 2   2020 Fall Salary List Club  800 non-null    object 
 3   Playing Position            798 non-null    object 
 4   Base Salary                 800 non-null    object 
 5   Unnamed: 5                  35 non-null     float64
 6   Base Guaranteed Comp.       800 non-null    object 
dtypes: float64(1), object(6)
memory usage: 43.9+ KB


In [24]:
salary2020_df.drop(columns='Unnamed: 5', inplace=True)

In [25]:
salary2020_df.head()

,Last Name,First Name,2020 Fall Salary List Club,Playing Position,Base Salary,Base Guaranteed Comp.
0,Aaronson,Brenden,Philadelphia Union,M,85000,103309
1,Abdul-Salaam,Saad,FC Cincinnati,D,81375,90042
2,Abu,Mohammed,DC United,M,81375,81375
3,Abubakar,Lalas,Colorado Rapids,D,145000,154937
4,Accam,David,Nashville SC,F,875000,1002916


In [26]:
# Looks like the column names have been inputted as values in the dataset. When checking the original spreadsheet,
# each page puts the column names at the top. These were converted into rows with the rest of the data accidentally. This
# likely happened with the other dataframes as well.
salary2020_df.describe()

,Last Name,First Name,2020 Fall Salary List Club,Playing Position,Base Salary,Base Guaranteed Comp.
count,795,800,800,798,800,800
unique,705,506,29,9,233,539
top,Last Name,First Name,Nashville SC,M,81375,81375
freq,22,22,33,245,140,89


In [27]:
# Column names are different and ordered differently, will have to resort. Fortunately, no extra columns.
salary2019_df.head()

,Club,Last Name,First Name,Playing Position,CY Salary (Annual),CY Guaranteed Comp (Annual)
0,Philadelphia Union,Aaronson,Brenden,M-F,70000.08,98309.48
1,Seattle Sounders FC,Abdul-Salaam,Saad,D,70250.04,70250.04
2,Colorado Rapids,Abubakar,Lalas,D,135000.00,144937.50
3,Columbus Crew,Accam,David,M-F,1010004.00,1137920.00
4,Colorado Rapids,Acosta,Kellyn,D-M,549999.96,664999.96


In [28]:
# Interesting, some First name nulls. 
salary2019_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714 entries, 0 to 713
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Club                         714 non-null    object 
 1   Last Name                    714 non-null    object 
 2   First Name                   705 non-null    object 
 3   Playing Position             713 non-null    object 
 4   CY Salary (Annual)           714 non-null    float64
 5   CY Guaranteed Comp (Annual)  714 non-null    float64
dtypes: float64(2), object(4)
memory usage: 33.6+ KB


In [29]:
# Ahh yes, Soccer players love to be known by only their last name. Otherwise, Nani would be known as 
# Luís Carlos Almeida da Cunha ComM (ComM is a Portugese honorific for great acts of service)
# Will have to deal with the nulls somehow and combine first and last name to player to blend with 
# statistics data.
salary2019_df[salary2019_df['First Name'].isna()]

,Club,Last Name,First Name,Playing Position,CY Salary (Annual),CY Guaranteed Comp (Annual)
45,Columbus Crew,Artur,NaN,D-M,360000.00,411633.33
51,Toronto FC,Auro,NaN,D,225000.00,276666.67
100,FC Dallas,Bressan,NaN,D,485300.04,521931.29
304,Philadelphia Union,Ilsinho,NaN,M,330000.00,357000.00
324,San Jose Earthquakes,Judson,NaN,D-M,300000.00,305000.00
326,LA Galaxy,Juninho,NaN,M,150000.00,163125.00
328,New York Red Bulls,Kaku,NaN,M-F,799999.92,799999.92
469,Orlando City SC,Nani,NaN,M-F,2333333.04,2486249.70
559,Orlando City SC,Robinho,NaN,M-F,198000.00,205500.00


In [30]:
# Looks like the column names didnt get added here 
salary2019_df.describe()

,CY Salary (Annual),CY Guaranteed Comp (Annual)
count,7.140000e+02,7.140000e+02
mean,3.715375e+05,4.119960e+05
std,6.275161e+05,7.071375e+05
min,5.624460e+04,5.625000e+04
25%,7.087500e+04,7.756959e+04
50%,1.637500e+05,1.794980e+05
75%,4.537530e+05,4.985530e+05
max,7.200000e+06,7.200000e+06


In [31]:
# Again, difference in column names
salary2018_df.head()

,Club,Last Name,First Name,Pos,Salary,Compensation
0,New York Red Bulls,Abang,Anatole,F,68927,68927
1,New York City FC,Abdul-Salaam,Saad,D,106480,120230
2,Columbus Crew,Abu,Mohammed,M,175008,181258
3,Columbus Crew,Abubakar,Lalas,D,68250.04,75750.04
4,Philadelphia Union,Accam,David,F-M,1250000.04,1250000.04


In [32]:
# one First name null
salary2018_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705 entries, 0 to 704
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Club           696 non-null    object
 1   Last Name      705 non-null    object
 2   First Name     704 non-null    object
 3   Pos            705 non-null    object
 4   Salary         705 non-null    object
 5   Compensation   705 non-null    object
dtypes: object(6)
memory usage: 33.2+ KB


In [33]:
# Column names in data
salary2018_df.describe()

,Club,Last Name,First Name,Pos,Salary,Compensation
count,696,705,704,705,705,705
unique,26,647,445,9,294,519
top,FC Dallas,Last Name,Chris,M,67500,67500
freq,34,11,15,230,72,49


In [34]:
#difference in column names
salary2017_df.head()

,Club,Last Name,First Name,Pos,Base Salary,Compensation
0,NYRB,Abang,Anatole,F,65625,65625
1,KC,Abdul-Salaam,Saad,D,96800,110550
2,CLB,Abu,Mohammed,M,165000,171250
3,CLB,Abubakar,"Alhassan ""Lalas""",D,65000.04,72500.04
4,CHI,Accam,David,F-M,750000,820937.5


In [35]:
# First name and Last name nulls
salary2017_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678 entries, 0 to 677
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Club          666 non-null    object
 1   Last Name     666 non-null    object
 2   First Name    661 non-null    object
 3   Pos           666 non-null    object
 4   Base Salary   678 non-null    object
 5   Compensation  678 non-null    object
dtypes: object(6)
memory usage: 31.9+ KB


In [36]:
#column names in data
salary2017_df.describe()

,Club,Last Name,First Name,Pos,Base Salary,Compensation
count,666,666,661,666,678,678
unique,25,604,406,11,268,457
top,VAN,Last Name,Chris,M,53004,53004
freq,34,12,14,227,52,43


In [37]:
salary2016_df.head()

,Club,Last Name,First Name,Pos,Base Salary,Compensation
0,NYRB,Abang,Anatole,F,62500,62500
1,KC,Abdul-Salaam,Saad,D,63000,76750
2,CHI,Accam,David,F-M,700000,770937.5
3,RSL,Acosta,Danilo,M,62500,62500
4,DAL,Acosta,Kellyn,M,220000,240000


In [38]:
# First and Last name nulls
salary2016_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Club          592 non-null    object
 1   Last Name     597 non-null    object
 2   First Name    596 non-null    object
 3   Pos           597 non-null    object
 4   Base Salary   608 non-null    object
 5   Compensation  608 non-null    object
dtypes: object(6)
memory usage: 28.6+ KB


In [39]:
# column names in data
salary2016_df.describe()

,Club,Last Name,First Name,Pos,Base Salary,Compensation
count,592,597,596,597,608,608
unique,23,538,373,11,210,401
top,DC,Last Name,Chris,M,63000,63000
freq,31,11,14,195,39,31


In [1]:
salary2015_df.head()

NameError: name 'salary2015_df' is not defined

In [41]:
# First and last name nulls
salary2015_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594 entries, 0 to 593
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Club          580 non-null    object
 1   Last Name     584 non-null    object
 2   First Name    580 non-null    object
 3   Pos           583 non-null    object
 4   Base Salary   594 non-null    object
 5   Compensation  594 non-null    object
dtypes: object(6)
memory usage: 28.0+ KB


In [42]:
# column names in data
salary2015_df.describe()

,Club,Last Name,First Name,Pos,Base Salary,Compensation
count,580,584,580,583,594,594
unique,22,533,372,15,175,353
top,SEA,Last Name,Chris,M,60000,60000
freq,32,10,13,198,149,93


In [43]:
salary2014_df.head()

,Club,Last Name,First Name,Pos,Base Salary,Compensation
0,DAL,Acosta,Kellyn,M,55000,79000
1,VAN,Adekugbe,Samuel,D,50000,55000
2,POR,Adi,Fanendo,F,580800,593300
3,COL,Agbossoumonde,Gale,D,48825,55491.67
4,DAL,Akindele,Tesho,F,48500,48500


In [44]:
# First and last name nulls
salary2014_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592 entries, 0 to 591
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Club          579 non-null    object
 1   Last Name     582 non-null    object
 2   First Name    578 non-null    object
 3   Pos           582 non-null    object
 4   Base Salary   592 non-null    object
 5   Compensation  592 non-null    object
dtypes: object(6)
memory usage: 27.9+ KB


In [45]:
# columns names in data
salary2014_df.describe()

,Club,Last Name,First Name,Pos,Base Salary,Compensation
count,579,582,578,582,592,592
unique,23,530,358,15,178,365
top,MTL,Last Name,Chris,M,48500,48500
freq,32,10,13,181,49,35


In [46]:
salary2013_df.head()

,Club,Last Name,First Name,Pos,Base Salary,Compensation
0,VAN,Abdallah,Aminu,M,46500,46500
1,DAL,Acosta,Kellyn,M,46500,70500
2,VAN,Adekugbe,Samuel,D,46500,51500
3,TOR,Agbossoumonde,Gale,D,46500,53166.67
4,NE,Agudelo,Juan,F,145000,175000


In [47]:
# First and last name nulls
salary2013_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596 entries, 0 to 595
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Club          583 non-null    object
 1   Last Name     586 non-null    object
 2   First Name    584 non-null    object
 3   Pos           586 non-null    object
 4   Base Salary   596 non-null    object
 5   Compensation  596 non-null    object
dtypes: object(6)
memory usage: 28.1+ KB


In [48]:
# column names in data
salary2013_df.describe()

,Club,Last Name,First Name,Pos,Base Salary,Compensation
count,583,586,584,586,596,596
unique,21,530,373,15,191,369
top,CHI,Last Name,Michael,M,46500,46500
freq,33,10,12,192,104,76


In [49]:
salary2012_df.head()

,Club,Last Name,First Name,Pos,Base Salary,Compensation
0,PHI,Adu,Freddy,M,400000,519000
1,CHV,Agudelo,Juan,F,70000,100000
2,KC,Aiyegbusi,Olukorede,D,44100,44100
3,COL,Akpan,Andre,F,57200,57200
4,PHI,Albright,Chris,D,55000,55000


In [50]:
#First and last name nulls
salary2012_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 574 entries, 0 to 573
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Club          562 non-null    object
 1   Last Name     564 non-null    object
 2   First Name    562 non-null    object
 3   Pos           564 non-null    object
 4   Base Salary   574 non-null    object
 5   Compensation  574 non-null    object
dtypes: object(6)
memory usage: 27.0+ KB


In [51]:
# Column names in data
salary2012_df.describe()

,Club,Last Name,First Name,Pos,Base Salary,Compensation
count,562,564,562,564,574,574
unique,22,515,352,14,176,367
top,POR,Last Name,Chris,M,44000,33750
freq,32,10,13,188,53,39


In [52]:
# This didnt work.
salary2018_df.rename(columns={'Salary': 'Base Salary'}, inplace=True)
salary2018_df

,Club,Last Name,First Name,Pos,Salary,Compensation
0,New York Red Bulls,Abang,Anatole,F,68927,68927
1,New York City FC,Abdul-Salaam,Saad,D,106480,120230
2,Columbus Crew,Abu,Mohammed,M,175008,181258
3,Columbus Crew,Abubakar,Lalas,D,68250.04,75750.04
4,Philadelphia Union,Accam,David,F-M,1250000.04,1250000.04
...,...,...,...,...,...,...
700,FC Dallas,Ziegler,Reto,D,750000,800000
701,LAFC,Zimmerman,Walker,D,235000,235000
702,Atlanta United,Zizzo,Sal,D,129999.96,129999.96
703,FC Dallas,Zobeck,Kyle,GK,67500,67500


In [53]:
# create a dicitonary to iterate over all dataframes 
salary_dict = {"2007": salary2007_df, "2008": salary2008_df, "2009": salary2009_df, "2010": salary2010_df, 
               "2011": salary2011_df, "2012": salary2012_df, "2013": salary2013_df, "2014": salary2014_df, 
               "2015": salary2015_df, "2016": salary2016_df, "2017": salary2017_df, "2018": salary2018_df,
               "2019": salary2019_df, "2020": salary2020_df}

In [54]:
# When attempting to call certain column names, received errors saying they didnt exist. 
# Strpping white space to try to fix this
for year in salary_dict.keys():
    salary_dict[year].columns = salary_dict[year].columns.str.strip()

In [55]:
# Renaming columns for consistency
salary2018_df.rename(columns={'Salary': 'Base Salary'}, inplace=True)
salary2018_df

,Club,Last Name,First Name,Pos,Base Salary,Compensation
0,New York Red Bulls,Abang,Anatole,F,68927,68927
1,New York City FC,Abdul-Salaam,Saad,D,106480,120230
2,Columbus Crew,Abu,Mohammed,M,175008,181258
3,Columbus Crew,Abubakar,Lalas,D,68250.04,75750.04
4,Philadelphia Union,Accam,David,F-M,1250000.04,1250000.04
...,...,...,...,...,...,...
700,FC Dallas,Ziegler,Reto,D,750000,800000
701,LAFC,Zimmerman,Walker,D,235000,235000
702,Atlanta United,Zizzo,Sal,D,129999.96,129999.96
703,FC Dallas,Zobeck,Kyle,GK,67500,67500


In [56]:
salary_dict["2018"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705 entries, 0 to 704
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Club          696 non-null    object
 1   Last Name     705 non-null    object
 2   First Name    704 non-null    object
 3   Pos           705 non-null    object
 4   Base Salary   705 non-null    object
 5   Compensation  705 non-null    object
dtypes: object(6)
memory usage: 33.2+ KB


In [57]:
salary2019_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714 entries, 0 to 713
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Club                         714 non-null    object 
 1   Last Name                    714 non-null    object 
 2   First Name                   705 non-null    object 
 3   Playing Position             713 non-null    object 
 4   CY Salary (Annual)           714 non-null    float64
 5   CY Guaranteed Comp (Annual)  714 non-null    float64
dtypes: float64(2), object(4)
memory usage: 33.6+ KB


In [58]:
#Renaming columns for consistency
salary2019_df.rename(columns={'Playing Position': 'Pos', 'CY Salary (Annual)': 'Base Salary',
                              'CY Guaranteed Comp (Annual)': 'Compensation'}, inplace=True)

In [59]:
salary2020_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Last Name                   795 non-null    object
 1   First Name                  800 non-null    object
 2   2020 Fall Salary List Club  800 non-null    object
 3   Playing Position            798 non-null    object
 4   Base Salary                 800 non-null    object
 5   Base Guaranteed Comp.       800 non-null    object
dtypes: object(6)
memory usage: 37.6+ KB


In [60]:
#Renaming columns for consistency
salary2020_df.rename(columns={'Playing Position': 'Pos', '2020 Fall Salary List Club': 'Club'}, inplace=True)

In [61]:
#Starting year at 2012, same as our statistics dataframes
df_year = 2007
for year in salary_dict.keys():
    # adding year column to each dataframe in salary_dict
    salary_dict.get(year, {})['year'] = df_year
    df_year+=1
    

In [62]:
salary2007_df.head()

,Club,Last Name,First Name,Pos,Base Salary,Compensation,year
0,DC,Addlery,Nicholas,F,36000,36000,2007
1,LA,Albright,Chris,M-D,142500,142500,2007
2,NY,Altidore,Josmer,F,85000,108333,2007
3,DAL,Alvarez,Arturo,M,32819,46019,2007
4,NY,Angel,Juan Pablo,F,1500000,1593750,2007


In [63]:
salary2015_df.head()

,Club,Last Name,First Name,Pos,Base Salary,Compensation,year
0,NY,Abang,Anatole,F,50000,50000,2015
1,KC,Abdul-Salaam,Saad,D,60000,73750,2015
2,CHI,Accam,David,F,650000,720937.5,2015
3,DAL,Acosta,Kellyn,M,60000,84000,2015
4,VAN,Adekugbe,Samuel,D,60000,65000,2015


In [64]:
# Rearranging order of columns for consistency
salary2020_df = pd.DataFrame(salary2020_df, columns=['Club', 'Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'year'])
salary2020_df

,Club,Last Name,First Name,Pos,Base Salary,Base Guaranteed Comp.,year
0,Philadelphia Union,Aaronson,Brenden,M,85000,103309,2020
1,FC Cincinnati,Abdul-Salaam,Saad,D,81375,90042,2020
2,DC United,Abu,Mohammed,M,81375,81375,2020
3,Colorado Rapids,Abubakar,Lalas,D,145000,154937,2020
4,Nashville SC,Accam,David,F,875000,1002916,2020
...,...,...,...,...,...,...,...
795,New York Red Bulls,NaN,Kaku,M,850000,850000,2020
796,Orlando City SC,NaN,Robinho,M,198000,205500,2020
797,Philadelphia Union,NaN,Ilsinho,M,425000,476250,2020
798,San Jose Earthquakes,NaN,Judson,M,320000,325000,2020


In [65]:
for year, df in salary_dict.items():
    # Remove columns i dont need
    salary_dict[year] = salary_dict[year].drop(['Club'], axis=1)

In [66]:
salary_dict['2012'].head()

,Last Name,First Name,Pos,Base Salary,Compensation,year
0,Adu,Freddy,M,400000,519000,2012
1,Agudelo,Juan,F,70000,100000,2012
2,Aiyegbusi,Olukorede,D,44100,44100,2012
3,Akpan,Andre,F,57200,57200,2012
4,Albright,Chris,D,55000,55000,2012


In [67]:
# Renaming columns to remove issue where columns cant be called because of hidden characters.
salary_dict['2007'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']
salary_dict['2008'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']
salary_dict['2009'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']
salary_dict['2010'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']
salary_dict['2011'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']
salary_dict['2012'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']
salary_dict['2013'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']
salary_dict['2014'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']
salary_dict['2015'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']
salary_dict['2016'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']
salary_dict['2017'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']
salary_dict['2018'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']
salary_dict['2019'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']
salary_dict['2020'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']

In [68]:
# Now that all salary dataframes are consistent, we can combine into one dataframe vertically.
salary_df = pd.concat(salary_dict, ignore_index=True)  
salary_df

,Last Name,First Name,Pos,Base Salary,Base Guaranteed Comp.,Year
0,Addlery,Nicholas,F,36000,36000,2007
1,Albright,Chris,M-D,142500,142500,2007
2,Altidore,Josmer,F,85000,108333,2007
3,Alvarez,Arturo,M,32819,46019,2007
4,Angel,Juan Pablo,F,1500000,1593750,2007
...,...,...,...,...,...,...
8007,NaN,Kaku,M,850000,850000,2020
8008,NaN,Robinho,M,198000,205500,2020
8009,NaN,Ilsinho,M,425000,476250,2020
8010,NaN,Judson,M,320000,325000,2020


In [69]:
# GK must be split because they have different statistics data.
salary_df['Pos'].value_counts()

Pos
M                   2449
D                   2242
F                   1563
GK                   883
M-F                  298
D-M                  178
Pos                  110
M-D                   74
F-M                   63
Playing Position      22
M/F                   20
M/D                    9
MF                     7
D/M                    6
F/M                    6
D/F                    4
F-D                    1
D-F                    1
Name: count, dtype: int64

In [70]:
# Creating player_salary_df without GK data
player_salary_df = salary_df[salary_df['Pos'] != 'GK']
player_salary_df

,Last Name,First Name,Pos,Base Salary,Base Guaranteed Comp.,Year
0,Addlery,Nicholas,F,36000,36000,2007
1,Albright,Chris,M-D,142500,142500,2007
2,Altidore,Josmer,F,85000,108333,2007
3,Alvarez,Arturo,M,32819,46019,2007
4,Angel,Juan Pablo,F,1500000,1593750,2007
...,...,...,...,...,...,...
8007,NaN,Kaku,M,850000,850000,2020
8008,NaN,Robinho,M,198000,205500,2020
8009,NaN,Ilsinho,M,425000,476250,2020
8010,NaN,Judson,M,320000,325000,2020


In [71]:
# Dropping rows where both First and Last name are null
player_salary_df.dropna(subset=['First Name', 'Last Name'], how='all', inplace=True)

/var/folders/g_/5gwjpx2j5c7cg2mn_rrdzqpc0000gn/T/ipykernel_11691/2983055035.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_salary_df.dropna(subset=['First Name', 'Last Name'], how='all', inplace=True)


In [72]:
player_salary_df.isna().sum()

Last Name                 5
First Name               29
Pos                       4
Base Salary               0
Base Guaranteed Comp.     0
Year                      0
dtype: int64

In [73]:
player_salary_df[player_salary_df['First Name'].isna()]

,Last Name,First Name,Pos,Base Salary,Base Guaranteed Comp.,Year
1852,Jeferson,NaN,F,429996,484996,2011
2157,Alex,NaN,M,96000,105950,2012
2409,Juninho,NaN,M,50000,65625,2012
2734,Alex,NaN,M,110000,119950,2013
3003,Juninho,NaN,M,290000,290000,2013
3328,Alex,NaN,M,123750,133700,2014
3487,Fabinho,NaN,D,96000,100500,2014
3592,Juninho,NaN,M,325000,325000,2014
3599,Kaka,NaN,M,6660000,7167500,2014
3924,Alex,NaN,M,142000,142000,2015


In [74]:
# Had to get creative, just making first and last name the same for players that only have one to be able to merge 
# first and last name
for i, player in player_salary_df.iterrows():
    if pd.isnull(player['First Name']):
        player_salary_df.at[i, 'First Name'] = player_salary_df.at[i, 'Last Name']
    if pd.isnull(player['Last Name']):
        player_salary_df.at[i, 'Last Name'] = player_salary_df.at[i, 'First Name']

In [75]:
player_salary_df[player_salary_df['First Name'].isna()]

,Last Name,First Name,Pos,Base Salary,Base Guaranteed Comp.,Year


In [76]:

for i, player in player_salary_df.iterrows():
    if player_salary_df.at[i, 'First Name'] == player_salary_df.at[i, 'Last Name']:
        # For rows where first and last name are the same, player name equals just the last name
        player_salary_df.at[i, 'Player'] = player_salary_df.at[i, 'Last Name']
    else:
        # player name equals first name plus last name for all other players 
        player_salary_df.at[i, 'Player'] = player_salary_df.at[i, 'First Name'] + " " +  player_salary_df.at[i, 'Last Name']
# Drop first name, last name, and position
player_salary_df.drop(columns=['First Name', 'Last Name', 'Pos'], inplace=True)

/var/folders/g_/5gwjpx2j5c7cg2mn_rrdzqpc0000gn/T/ipykernel_11691/644187849.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_salary_df.at[i, 'Player'] = player_salary_df.at[i, 'First Name'] + " " +  player_salary_df.at[i, 'Last Name']
/var/folders/g_/5gwjpx2j5c7cg2mn_rrdzqpc0000gn/T/ipykernel_11691/644187849.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Nicholas Addlery' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  player_salary_df.at[i, 'Player'] = player_salary_df.at[i, 'First Name'] + " " +  player_salary_df.at[i, 'Last Name']
/var/folders/g_/5gwjpx2j5c7cg2mn_rrdzqpc0000gn/T/ipykernel_11691/644187849.py:9: Settin

In [77]:
player_salary_df

,Base Salary,Base Guaranteed Comp.,Year,Player
0,36000,36000,2007,Nicholas Addlery
1,142500,142500,2007,Chris Albright
2,85000,108333,2007,Josmer Altidore
3,32819,46019,2007,Arturo Alvarez
4,1500000,1593750,2007,Juan Pablo Angel
...,...,...,...,...
8007,850000,850000,2020,Kaku
8008,198000,205500,2020,Robinho
8009,425000,476250,2020,Ilsinho
8010,320000,325000,2020,Judson


In [78]:
#Create keeper_salary_df, containing data for only GK
keeper_salary_df = salary_df[salary_df['Pos'] == 'GK']
keeper_salary_df

,Last Name,First Name,Pos,Base Salary,Base Guaranteed Comp.,Year
33,Burpo,Preston,GK,52491.6,52491.6,2007
34,Burse,Ray,GK,17700,17700,2007
35,Busch,Jon,GK,58008,58008,2007
41,Cannon,Joe,GK,192000,192000,2007
48,Cepero,Danny,GK,12900,12900,2007
...,...,...,...,...,...,...
7973,Westberg,Quentin,GK,240000,253333,2020
7979,Willis,Joe,GK,180000,187833,2020
7989,Yarbrough,William,GK,120000,120000,2020
7999,Zendejas,Adrian,GK,81375,81375,2020


In [79]:
# Keeper player name  equals first name plus last name
keeper_salary_df['Player'] = keeper_salary_df['First Name'] + " " + keeper_salary_df['Last Name']
# drop first name, last name, and position
keeper_salary_df.drop(columns=['First Name', 'Last Name', 'Pos'], inplace=True)

/var/folders/g_/5gwjpx2j5c7cg2mn_rrdzqpc0000gn/T/ipykernel_11691/3645307494.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keeper_salary_df['Player'] = keeper_salary_df['First Name'] + " " + keeper_salary_df['Last Name']
/var/folders/g_/5gwjpx2j5c7cg2mn_rrdzqpc0000gn/T/ipykernel_11691/3645307494.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keeper_salary_df.drop(columns=['First Name', 'Last Name', 'Pos'], inplace=True)


In [80]:
# Creating a normalize_string function to clean Player column strings to combine stats  and salary dataframes on player and year
def normalize_string(string):
    normalized = unidecode(string)
    re.sub(r'[^A-Za-z0-9\s]+', '', string)
    return re.sub(r'[^\w\s]', '', normalized)

In [81]:
# apply function to player stats and salary df's
player_stats_df['Player'] = player_stats_df['Player'].apply(normalize_string)
player_salary_df['Player'] = player_salary_df['Player'].apply(normalize_string)
# change all strings to lowercase
player_stats_df['Player'] = player_stats_df['Player'].str.lower()
player_salary_df['Player'] = player_salary_df['Player'].str.lower()

/var/folders/g_/5gwjpx2j5c7cg2mn_rrdzqpc0000gn/T/ipykernel_11691/2746037855.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_stats_df['Player'] = player_stats_df['Player'].apply(normalize_string)
/var/folders/g_/5gwjpx2j5c7cg2mn_rrdzqpc0000gn/T/ipykernel_11691/2746037855.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_salary_df['Player'] = player_salary_df['Player'].apply(normalize_string)
/var/folders/g_/5gwjpx2j5c7cg2mn_rrdzqpc0000gn/T/ipykernel_11691/2746037855.py:5: SettingWithCopyW

In [82]:
#Create lists for comparison
left_player_name = []
fuzz_player_name = []
# Looping over each player name in stats_df to compare to every player name in salary_df using fuzz. Once the string with the
# highest similarity is found, stats_df player name is changed to the closest match.
for i, player in player_stats_df.iterrows():
    left_player = player['Player']
    left_player_name.append(left_player)
    closest_match = None
    highest_similarity = 0

    for _, right_player in player_salary_df.iterrows():
        right_player_1 = right_player['Player']
        similarity = fuzz.ratio(left_player, right_player_1)

        if similarity > highest_similarity:
            highest_similarity = similarity
            closest_match = right_player_1
            
    if highest_similarity < 80: 
        fuzz_player_name.append(left_player)
    else:
        fuzz_player_name.append(closest_match)
        player_stats_df.at[i, 'Player'] = closest_match

In [83]:
#creating a fuzz df to see how fuzz did
fuzz_df = pd.DataFrame({'Left Name': left_player_name,
                        'Fuzz Name': fuzz_player_name})
fuzz_df

,Left Name,Fuzz Name
0,luciano emilio,luciano emilio
1,juan pablo angel,juan pablo angel
2,taylor twellman,taylor twellman
3,eddie johnson,eddie johnson
4,maykel galindo,maykel galindo
...,...,...
7240,sebastian mendez,sebastian fernandez
7241,kelyn rowe,kelyn rowe
7242,siem de jong,siem de jong
7243,maxime chanot,maxime chanot


In [84]:
# creating a bad_fuzz df to see which names did not match up.
# Unfortunately it looks like there is potentially good data getting thrown in here but there are also bad matches. Throwing
# these datasets out will keep most of our data intact
bad_fuzz = fuzz_df[fuzz_df['Left Name'] != fuzz_df['Fuzz Name']]
bad_fuzz

,Left Name,Fuzz Name
22,dwayne de rosario,dwayne derosario
25,danny dichio,daniel dichio
26,jeff cunningham,jeffrey cunningham
75,jose burciaga jr,jose burciaga
99,andy welsh,andrew welsh
...,...,...
7160,aljaz struna,aljaz kiki struna
7198,jeizon ramirez,jeizon ramirez chacon
7221,miguel ibarra,miguel ibarra
7225,matias vera,matias jara


In [85]:
fuzz.ratio('sebastian mendez', 'sebastian fernandez')

85.71428571428572

In [86]:
# merging keeper stats and salary df's 
keeper_stats_salaries_df = pd.merge(keeper_stats_df, keeper_salary_df, how='left',
                                    left_on=['Player', 'Year'], right_on=['Player', 'Year'])
keeper_stats_salaries_df                    

,Player,Club,POS,GP,GS,MINS,SHTS,SV,GA,GAA,...,W,L,T,ShO,W%,Sv%,Year,Season,Base Salary,Base Guaranteed Comp.
0,Bouna Coundoul,COL,GK,30,30,2668,158,120,32,1.07,...,9,12,8,9,30.0,75.9,2007,reg,30000,30000
1,Kevin Hartman,KC,GK,30,30,2700,159,110,45,1.50,...,11,12,7,5,36.7,69.2,2007,reg,150000,150000
2,Matt Reis,NE,GK,30,30,2700,169,120,43,1.43,...,14,8,8,10,46.7,71.0,2007,reg,150000,150000
3,Joe Cannon,LA,GK,29,29,2610,171,119,46,1.59,...,9,13,7,5,31.0,69.6,2007,reg,192000,192000
4,Troy Perkins,DC,GK,29,29,2610,155,117,32,1.10,...,16,6,7,8,55.2,75.5,2007,reg,85000,87500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,Emmanuel Ochoa,SJ,GK,0,0,0,0,0,0,0.00,...,0,0,0,0,0.0,0.0,2020,reg,63547,73759
893,Drake Callender,MIA,GK,0,0,0,0,0,0,0.00,...,0,0,0,0,0.0,0.0,2020,reg,100000,100000
894,Elliot Panicco,NSH,GK,0,0,0,0,0,0,0.00,...,0,0,0,0,0.0,0.0,2020,reg,63547,63547
895,Eric Dick,SKC,GK,0,0,0,0,0,0,0.00,...,0,0,0,0,0.0,0.0,2020,reg,81375,85020


In [87]:
# Some nulls in Salary
keeper_stats_salaries_df.isna().sum()

Player                     0
Club                       0
POS                        0
GP                         0
GS                         0
MINS                       0
SHTS                       0
SV                         0
GA                         0
GAA                        0
PKG/A                      0
W                          0
L                          0
T                          0
ShO                        0
W%                         0
Sv%                        0
Year                       0
Season                     0
Base Salary              107
Base Guaranteed Comp.    107
dtype: int64

In [88]:
player_stats_salaries_df = pd.merge(player_stats_df, player_salary_df, how='left', 
                                    left_on=['Player', 'Year'], right_on=['Player', 'Year'])
player_stats_salaries_df

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,...,FC,FS,OFF,YC,RC,SOG%,Year,Season,Base Salary,Base Guaranteed Comp.
0,luciano emilio,DC,F,29,28,2410,20,1,79,47,...,35,39,21,2,0,59.49,2007,reg,265000,293125
1,juan pablo angel,NY,F,24,24,2125,19,5,97,53,...,31,20,42,2,1,54.64,2007,reg,1500000,1593750
2,taylor twellman,NE,F,26,25,2283,16,3,90,55,...,13,41,21,2,0,61.11,2007,reg,325008,350008
3,eddie johnson,KC,F,24,24,2149,15,6,75,43,...,21,57,50,6,0,57.33,2007,reg,750000,875000
4,maykel galindo,CHV,F,28,24,2021,12,5,55,28,...,44,53,30,6,0,50.91,2007,reg,72500,72500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7240,sebastian fernandez,ORL,M,19,11,1087,0,0,17,3,...,32,9,1,4,1,17.65,2020,reg,NaN,NaN
7241,kelyn rowe,NE,M,16,10,896,0,0,17,4,...,13,4,0,3,0,23.53,2020,reg,310610,310610
7242,siem de jong,CIN,M,15,8,793,0,0,18,5,...,7,8,1,0,0,27.78,2020,reg,1157556,1287556
7243,maxime chanot,NYC,D,20,20,1712,0,0,19,4,...,18,6,1,5,1,21.05,2020,reg,725000,725000


Club             9
Last Name        9
First Name      10
Pos              9
Base Salary      0
Compensation     0
year             0
dtype: int64

In [89]:
#Some nulls to deal with
player_stats_salaries_df.isna().sum()

Player                      0
Club                        0
POS                         0
GP                          0
GS                          0
MINS                        0
G                           0
A                           0
SHTS                        0
SOG                         0
GWG                         0
PKG/A                       0
HmG                         0
RdG                         0
G/90min                     0
SC%                         0
GWA                         0
HmA                         0
RdA                         0
A/90min                     0
FC                          0
FS                          0
OFF                         0
YC                          0
RC                          0
SOG%                     1528
Year                        0
Season                      0
Base Salary               777
Base Guaranteed Comp.     777
dtype: int64

In [90]:
player_stats_salaries_df[player_stats_salaries_df['Base Salary'].isna()].head(30)

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,...,FC,FS,OFF,YC,RC,SOG%,Year,Season,Base Salary,Base Guaranteed Comp.
6,christian gomez,DC,M,27,27,2272,10,9,82,44,...,34,43,10,6,0,53.66,2007,reg,NaN,NaN
7,jozy altidore,TOR,F,22,15,1399,9,4,43,20,...,38,43,12,4,0,46.51,2007,reg,NaN,NaN
9,robbie findley,LA,F,25,14,1353,8,0,31,16,...,15,27,14,0,0,51.61,2007,reg,NaN,NaN
10,fred,DC,M,26,23,2096,7,8,28,14,...,45,40,21,5,1,50.00,2007,reg,NaN,NaN
27,juan toja,DAL,M,27,27,2388,6,1,33,16,...,83,41,6,4,0,48.48,2007,reg,NaN,NaN
45,herculez gomez,COL,F,20,16,1537,4,2,51,22,...,38,46,17,6,0,43.14,2007,reg,NaN,NaN
65,roberto brown,COL,F,13,8,696,3,0,14,4,...,31,11,16,2,1,28.57,2007,reg,NaN,NaN
83,fabian espindola,RSL,F,12,8,602,2,2,13,9,...,7,16,7,3,1,69.23,2007,reg,NaN,NaN
109,freddy adu,RSL,F,11,10,899,1,2,24,12,...,11,25,6,1,0,50.00,2007,reg,NaN,NaN
116,willian oliveira,CHI,M,8,4,381,1,1,7,4,...,1,6,3,0,0,57.14,2007,reg,NaN,NaN


In [91]:
# Dropping all rows with player names in bad fuzz df
for i, player in player_stats_salaries_df.iterrows():
    if player['Player'] in bad_fuzz['Fuzz Name'].values:
        player_stats_salaries_df.drop(i, inplace=True)

In [92]:
player_stats_salaries_df.isna().sum()

Player                      0
Club                        0
POS                         0
GP                          0
GS                          0
MINS                        0
G                           0
A                           0
SHTS                        0
SOG                         0
GWG                         0
PKG/A                       0
HmG                         0
RdG                         0
G/90min                     0
SC%                         0
GWA                         0
HmA                         0
RdA                         0
A/90min                     0
FC                          0
FS                          0
OFF                         0
YC                          0
RC                          0
SOG%                     1415
Year                        0
Season                      0
Base Salary               728
Base Guaranteed Comp.     728
dtype: int64

In [93]:

player_stats_salaries_df[player_stats_salaries_df['Base Salary'].isna()]

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,...,FC,FS,OFF,YC,RC,SOG%,Year,Season,Base Salary,Base Guaranteed Comp.
6,christian gomez,DC,M,27,27,2272,10,9,82,44,...,34,43,10,6,0,53.66,2007,reg,NaN,NaN
7,jozy altidore,TOR,F,22,15,1399,9,4,43,20,...,38,43,12,4,0,46.51,2007,reg,NaN,NaN
9,robbie findley,LA,F,25,14,1353,8,0,31,16,...,15,27,14,0,0,51.61,2007,reg,NaN,NaN
10,fred,DC,M,26,23,2096,7,8,28,14,...,45,40,21,5,1,50.00,2007,reg,NaN,NaN
27,juan toja,DAL,M,27,27,2388,6,1,33,16,...,83,41,6,4,0,48.48,2007,reg,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7154,wilfried zahibo,HOU,M,7,3,298,0,0,2,0,...,15,3,1,2,0,0.00,2020,reg,NaN,NaN
7174,sam nicholson,COL,F,2,2,174,0,0,3,0,...,2,3,2,0,0,0.00,2020,reg,NaN,NaN
7223,aleksandar katai,LA,M,2,2,117,0,0,8,0,...,5,2,0,2,0,0.00,2020,reg,NaN,NaN
7236,boniek garcia,HOU,M,18,7,819,0,0,12,1,...,12,9,0,5,0,8.33,2020,reg,NaN,NaN


In [94]:
# Drop all rows where Base Salary is null
player_stats_salaries_df.dropna(subset='Base Salary',inplace = True)

In [95]:
player_stats_salaries_df.isna().sum()

Player                      0
Club                        0
POS                         0
GP                          0
GS                          0
MINS                        0
G                           0
A                           0
SHTS                        0
SOG                         0
GWG                         0
PKG/A                       0
HmG                         0
RdG                         0
G/90min                     0
SC%                         0
GWA                         0
HmA                         0
RdA                         0
A/90min                     0
FC                          0
FS                          0
OFF                         0
YC                          0
RC                          0
SOG%                     1136
Year                        0
Season                      0
Base Salary                 0
Base Guaranteed Comp.       0
dtype: int64

In [96]:
player_stats_salaries_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6161 entries, 0 to 7243
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Player                 6161 non-null   object 
 1   Club                   6161 non-null   object 
 2   POS                    6161 non-null   object 
 3   GP                     6161 non-null   int64  
 4   GS                     6161 non-null   int64  
 5   MINS                   6161 non-null   int64  
 6   G                      6161 non-null   int64  
 7   A                      6161 non-null   int64  
 8   SHTS                   6161 non-null   int64  
 9   SOG                    6161 non-null   int64  
 10  GWG                    6161 non-null   int64  
 11  PKG/A                  6161 non-null   object 
 12  HmG                    6161 non-null   int64  
 13  RdG                    6161 non-null   int64  
 14  G/90min                6161 non-null   float64
 15  SC%      

In [97]:
keeper_stats_salaries_df.isna().sum()

Player                     0
Club                       0
POS                        0
GP                         0
GS                         0
MINS                       0
SHTS                       0
SV                         0
GA                         0
GAA                        0
PKG/A                      0
W                          0
L                          0
T                          0
ShO                        0
W%                         0
Sv%                        0
Year                       0
Season                     0
Base Salary              107
Base Guaranteed Comp.    107
dtype: int64

In [98]:
#Drop all rows where Base salary is null
keeper_stats_salaries_df.dropna(subset='Base Salary', inplace=True)

In [99]:
#convert to csv 
player_stats_salaries_df.to_csv('./data/player_stats_salaries.csv')
keeper_stats_salaries_df.to_csv('./data/keeper_stats_salaries.csv')